# Apply DESI footprint to full sky catalogues (galaxy type 3 -- ELG)

In [1]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
from desimodel.io import fits
import glob
import os

# these four parameters are for modeling n(z)
mu = -1.204
sigma =1
shift =0.6836
amp =0.0006036407796*1.63

for i in range(24,71):
 ###############################   
 ### change input files here ###
    CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype3_{}.fits".format(i)
    d        = fits.open(CAT_file)
    # compute the number density of the given snapshot by total number of galaxies / volume
    n_mean=d[0].header['NGALBOX']/np.power(10,9)

    ra0       = d[1].data['RA']
    dec0      = d[1].data['DEC']
    zz0       = d[1].data['Z']
    zz_rsd0   = d[1].data['Z']+d[1].data['DZ']

    ra1       = ra0[zz0>shift]
    dec1      = dec0[zz0>shift]
    zz1       = zz0[zz0>shift]
    zz_rsd1   = zz_rsd0[zz0>shift]

    # n(z) model
    nz=amp*np.exp(-(np.log(zz1-shift)-mu)**2/(2*sigma**2))/((zz1-shift)*sigma*np.sqrt(2*np.pi))
    # downsample
    ran=np.random.rand(len(ra1))
    nz_selected=(ran<nz/n_mean)
    index1=[]
    index1=np.where(nz_selected)
    ra       = ra1[index1]
    dec      = dec1[index1]
    zz       = zz1[index1]
    zz_rsd   = zz_rsd1[index1]
    nzz = amp*np.exp(-(np.log(zz-shift)-mu)**2/(2*sigma**2))/((zz-shift)*sigma*np.sqrt(2*np.pi))
    
    print(f"Applying footprint for file {i}")
    tiles = desimodel.io.load_tiles()
    point=foot.is_point_in_desi(tiles, ra, dec)
    index=[]
    index = np.where(point)
    ra_desi=ra[index]
    dec_desi=dec[index]
    zz_desi=zz[index]
    zz_rsd_desi=zz_rsd[index]
    nzz_desi=nzz[index]
    
    print(len(ra))
    print(len(ra0))
    print(len(ra_desi))
    print(zz0[1])
    print('Writing footprinted catalog')
    
    ############################################ 
    ### change the name of output files here ###
    OUT_file ="/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/lightcone_multibox_galtype3_{}_footprint_nz.dat".format(i)
    np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi,nzz_desi])
    print(datetime.datetime.now().time())
    
# merge the output files above    
#######################################################    
### change the name to be merged to be consistent with the name above ### 
### change the name of the merged file ###
for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/lightcone_multibox_galtype3_*_footprint_nz.dat"):
         os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/single_file/UNIT_lightcone_multibox_ELG_footprint_nz.dat")
print('Done!')    

Selecting footprint 24
0
3460477
0
0.46892738006248313
Writing footprinted catalog
19:05:31.993224
Selecting footprint 25
0
3673277
0
0.4913777207206076
Writing footprinted catalog
19:05:32.331152
Selecting footprint 26
0
3983486
0
0.5127138315097408
Writing footprinted catalog
19:05:32.710563
Selecting footprint 27
0
4136887
0
0.5360470394993806
Writing footprinted catalog
19:05:33.073051
Selecting footprint 28
0
4473191
0
0.5587372786028673
Writing footprinted catalog
19:05:33.423896
Selecting footprint 29
0
4789460
0
0.5815331399482218
Writing footprinted catalog
19:05:33.828567
Selecting footprint 30
0
5126663
0
0.6048005595369272
Writing footprinted catalog
19:05:34.190310
Selecting footprint 31
0
5406850
0
0.6275203910344325
Writing footprinted catalog
19:05:34.658954
Selecting footprint 32
0
5764005
0
0.6512999491450333
Writing footprinted catalog
19:05:35.057018
Selecting footprint 33
0
6273703
0
0.675108303240276
Writing footprinted catalog
19:05:36.188351
Selecting footprint 

# Combine all the footprint-ed files to a single one (if not been done correctly in the previous step

In [5]:
import glob
import os
for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/lightcone_multibox_galtype3_*_footprint.dat"):
         os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/single_file/UNIT_lightcone_multibox_ELG_footprint.dat")

# Making randoms: replace all the ra dec by random positions, keep the redshifts from data

In [5]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
import glob
import os
from desimodel.io import fits

mu = -1.204
sigma =1
shift =0.6836
amp =0.0006036407796*1.63

## make random of 5x data ##
for j in range(1,6):
    for i in range(24,71):
 ###############################   
 ### change input files here ###
        CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype3_{}.fits".format(i)
        d        = fits.open(CAT_file)
        n_mean=d[0].header['NGALBOX']/np.power(10,9)
        length = d[1].header['NAXIS2']
    #    ra       = d[1].data['RA']
        ra0 = np.random.rand(length)*360
    #    dec      = d[1].data['DEC']
        dec0 = np.arccos(2*np.random.rand(length)-1)*90/np.pi*2-90
        zz0       = d[1].data['Z']
        zz_rsd0   = d[1].data['Z']+d[1].data['DZ']
        
        ra1       = ra0[zz0>shift]
        dec1      = dec0[zz0>shift]
        zz1       = zz0[zz0>shift]
        zz_rsd1   = zz_rsd0[zz0>shift]

        nz=amp*np.exp(-(np.log(zz1-shift)-mu)**2/(2*sigma**2))/((zz1-shift)*sigma*np.sqrt(2*np.pi))
        ran=np.random.rand(len(ra1))
        nz_selected=(ran<nz/n_mean)
        index1=[]
        index1=np.where(nz_selected)
        ra       = ra1[index1]
        dec      = dec1[index1]
        zz       = zz1[index1]
        zz_rsd   = zz_rsd1[index1]   
        nzz = amp*np.exp(-(np.log(zz-shift)-mu)**2/(2*sigma**2))/((zz-shift)*sigma*np.sqrt(2*np.pi))
    
        print(f"Applying footprint for file {i} in set {j}")
        tiles = desimodel.io.load_tiles()
        point=foot.is_point_in_desi(tiles, ra, dec)
        index=[]
        index = np.where(point)
        ra_desi=ra[index]
        dec_desi=dec[index]
        zz_desi=zz[index]
        zz_rsd_desi=zz_rsd[index]
        nzz_desi=nzz[index]
        
        print('Writing footprinted catalog')
    ############################################ 
    ### change the name of output files here ###        
        OUT_file ="/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/random/lightcone_multibox_galtype3_{}_footprint_nz_{}.ran".format(i,j)
        np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi, nzz_desi])
        print(datetime.datetime.now().time())
        
    print(f"starting combine files {j}")
# merge the output files above    
#######################################################    
### change the name to be merged to be consistent with the name above ### 
### change the name of the merged file ###
    for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/random/lightcone_multibox_galtype3_*_footprint_nz_{}.ran".format(j)):
        os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/random/single_file/UNIT_lightcone_multibox_ELG_footprint_nz_1xdata_{}.ran".format(j))
## 5xdata ##
        os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/random/single_file/UNIT_lightcone_multibox_ELG_footprint_nz_5xdata.ran")
    print('finishing combine files')

Selecting footprint 24 in set 1
Writing footprinted catalog
13:02:27.090353
Selecting footprint 25 in set 1
Writing footprinted catalog
13:02:27.601906
Selecting footprint 26 in set 1
Writing footprinted catalog
13:02:28.199770
Selecting footprint 27 in set 1
Writing footprinted catalog
13:02:28.746775
Selecting footprint 28 in set 1
Writing footprinted catalog
13:02:29.384044
Selecting footprint 29 in set 1
Writing footprinted catalog
13:02:30.031526
Selecting footprint 30 in set 1
Writing footprinted catalog
13:02:30.721599
Selecting footprint 31 in set 1
Writing footprinted catalog
13:02:31.461492
Selecting footprint 32 in set 1
Writing footprinted catalog
13:02:32.211622
Selecting footprint 33 in set 1
Writing footprinted catalog
13:02:33.146972
Selecting footprint 34 in set 1
Writing footprinted catalog
13:02:35.886173
Selecting footprint 35 in set 1
Writing footprinted catalog
13:02:52.338662
Selecting footprint 36 in set 1
Writing footprinted catalog
13:03:16.917121
Selecting fo

KeyboardInterrupt: 

# Merge the random files into a single file -- I have included this step in previous step.

In [9]:
import glob
import os
for j in range(1,2):
    print(f"starting combine files {j}")
    for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/random/lightcone_multibox_galtype3_*_footprint_{}.ran".format(j)):
             os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_ELG_footprint_N_snapshot/random/single_file/UNIT_lightcone_multibox_ELG_footprint_1xdata_{}.ran".format(j))
    print('finishing combine files')

starting combine files 1
finishing combine files


# making multiple snapshot lightcone for galtype2 (LRG)

In [2]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
from desimodel.io import fits
import glob
import os

mu = -1.505
sigma =0.451
shift =0.5
amp =0.000176*2

for i in range(6,71):
 ###############################   
 ### change input files here ###    
    CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype2_{}.fits".format(i)
    d        = fits.open(CAT_file)
   
    n_mean=d[0].header['NGALBOX']/np.power(10,9)
    print(n_mean)

    ra0       = d[1].data['RA']
    dec0      = d[1].data['DEC']
    zz0       = d[1].data['Z']
    zz_rsd0   = d[1].data['Z']+d[1].data['DZ']

    ra1       = ra0[zz0>shift]
    dec1      = dec0[zz0>shift]
    zz1       = zz0[zz0>shift]
    zz_rsd1   = zz_rsd0[zz0>shift]
    
    nz=amp*np.exp(-(np.log(zz1-shift)-mu)**2/(2*sigma**2))/((zz1-shift)*sigma*np.sqrt(2*np.pi))
    ran=np.random.rand(len(ra1))
    nz_selected=(ran<nz/n_mean)
    index1=[]
    index1=np.where(nz_selected)
    ra       = ra1[index1]
    dec      = dec1[index1]
    zz       = zz1[index1]
    zz_rsd   = zz_rsd1[index1]
    nzz = amp*np.exp(-(np.log(zz-shift)-mu)**2/(2*sigma**2))/((zz-shift)*sigma*np.sqrt(2*np.pi))    

    print(f"Selecting footprint {i}")
    tiles = desimodel.io.load_tiles()
    point=foot.is_point_in_desi(tiles, ra, dec)
    index=[]
    index = np.where(point)
    ra_desi=ra[index]
    dec_desi=dec[index]
    zz_desi=zz[index]
    zz_rsd_desi=zz_rsd[index]
    nzz_desi=nzz[index]
    
    print('Writing footprinted catalog')
    ############################################ 
    ### change the name of output files here ### 
    OUT_file ="/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_LRG_footprint_N_snapshot/lightcone_multibox_galtype2_{}_footprint_nz.dat".format(i)
#    np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi])
    np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi,nzz_desi])
    print(datetime.datetime.now().time())
    
# merge the output files above    
#######################################################    
### change the name to be merged to be consistent with the name above ### 
### change the name of the merged file ###    
for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_LRG_footprint_N_snapshot/lightcone_multibox_galtype2_*_footprint_nz.dat"):
         os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_LRG_footprint_N_snapshot/single_file/UNIT_lightcone_multibox_LRG_footprint_nz.dat")
print('Done!')  

FileNotFoundError: [Errno 2] No such file or directory: '/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype2_6.fits'

# make randoms for LRG

In [3]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
import glob
import os
from desimodel.io import fits

for j in range(1,2):
    for i in range(24,71):
 ###############################   
 ### change input files here ### 
        CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype1_{}.fits".format(i)
        d        = fits.open(CAT_file)
        length = d[1].header['NAXIS2']
    #    ra       = d[1].data['RA']
        ra = np.random.rand(length)*360
    #    dec      = d[1].data['DEC']
        dec = np.arccos(2*np.random.rand(length)-1)*90/np.pi*2-90
        zz       = d[1].data['Z']
        zz_rsd   = d[1].data['Z']+d[1].data['DZ']
        print(f"Selecting footprint {i} in set {j}")
        tiles = desimodel.io.load_tiles()
        point=foot.is_point_in_desi(tiles, ra, dec)
        index=[]
        index = np.where(point)
        ra_desi=ra[index]
        dec_desi=dec[index]
        zz_desi=zz[index]
        zz_rsd_desi=zz_rsd[index]
        print('Writing footprinted catalog')
    ############################################ 
    ### change the name of output files here ### 
        OUT_file ="/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_LRG_footprint_N_snapshot/random/lightcone_multibox_galtype1_{}_footprint_{}.ran".format(i,j)
        np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi])
        print(datetime.datetime.now().time())
        
# merge the output files above    
#######################################################    
### change the name to be merged to be consistent with the name above ### 
### change the name of the merged file ###        
    print(f"starting combine files {j}")
    for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_LRG_footprint_N_snapshot/random/lightcone_multibox_galtype1_*_footprint_{}.ran".format(j)):
             os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_LRG_footprint_N_snapshot/random/single_file/UNIT_lightcone_multibox_LRG_footprint_1xdata_{}.ran".format(j))
    print('finishing combine files')

Selecting footprint 24 in set 1
Writing footprinted catalog
04:01:06.657206
Selecting footprint 25 in set 1
Writing footprinted catalog
04:01:07.294546
Selecting footprint 26 in set 1
Writing footprinted catalog
04:01:07.989286
Selecting footprint 27 in set 1
Writing footprinted catalog
04:01:08.674348
Selecting footprint 28 in set 1
Writing footprinted catalog
04:01:09.344055
Selecting footprint 29 in set 1
Writing footprinted catalog
04:01:09.999348
Selecting footprint 30 in set 1
Writing footprinted catalog
04:01:10.895587
Selecting footprint 31 in set 1
Writing footprinted catalog
04:01:11.714350
Selecting footprint 32 in set 1
Writing footprinted catalog
04:01:12.645059
Selecting footprint 33 in set 1
Writing footprinted catalog
04:01:13.920291
Selecting footprint 34 in set 1
Writing footprinted catalog
04:01:15.145441
Selecting footprint 35 in set 1
Writing footprinted catalog
04:01:16.122870
Selecting footprint 36 in set 1
Writing footprinted catalog
04:01:17.167510
Selecting fo

KeyboardInterrupt: 

# making multiple snapshot lightcone for galtype1 (QSO)

In [1]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
from desimodel.io import fits
import glob
import os

# model for QSO n(z)
a=-0.00001*1.65*0.6
b=0.000038*1.65*0.6

for i in range(24,71):
 ###############################   
 ### change input files here ### 
    CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype1_{}.fits".format(i)
    d        = fits.open(CAT_file)
    
    n_mean=d[0].header['NGALBOX']/np.power(10,9)
    print(n_mean)
    
    ra0       = d[1].data['RA']
    dec0      = d[1].data['DEC']
    zz0       = d[1].data['Z']
    zz_rsd0   = d[1].data['Z']+d[1].data['DZ']

    ra1       = ra0[zz0<(-b/a)]
    dec1      = dec0[zz0<(-b/a)]
    zz1       = zz0[zz0<(-b/a)]
    zz_rsd1   = zz_rsd0[zz0<(-b/a)]
    
#    nz=amp*np.exp(-(np.log(zz1-shift)-mu)**2/(2*sigma**2))/((zz1-shift)*sigma*np.sqrt(2*np.pi))
    nz = a*zz1 + b
    ran=np.random.rand(len(ra1))
    nz_selected=(ran<nz/n_mean)
    index1=[]
    index1=np.where(nz_selected)
    ra       = ra1[index1]
    dec      = dec1[index1]
    zz       = zz1[index1]
    zz_rsd   = zz_rsd1[index1]
#    nzz = amp*np.exp(-(np.log(zz-shift)-mu)**2/(2*sigma**2))/((zz-shift)*sigma*np.sqrt(2*np.pi))
    nzz = a*zz + b

    print(f"Selecting footprint {i}")
    tiles = desimodel.io.load_tiles()
    point=foot.is_point_in_desi(tiles, ra, dec)
    index=[]
    index = np.where(point)
    ra_desi=ra[index]
    dec_desi=dec[index]
    zz_desi=zz[index]
    zz_rsd_desi=zz_rsd[index]
    nzz_desi=nzz[index]
    
    print('Writing footprinted catalog')
    ############################################ 
    ### change the name of output files here ### 
    OUT_file ="/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_QSO_footprint_N_snapshot/lightcone_multibox_galtype1_{}_footprint_nz.dat".format(i)
    np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi])
    print(datetime.datetime.now().time())
    
# merge the output files above    
#######################################################    
### change the name to be merged to be consistent with the name above ### 
### change the name of the merged file ###    
for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_QSO_footprint_N_snapshot/lightcone_multibox_galtype1_*_footprint_nz.dat"):
         os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_QSO_footprint_N_snapshot/single_file/UNIT_lightcone_multibox_QSO_footprint_nz.dat")
print('Done')

8.8836e-05
Selecting footprint 24
Writing footprinted catalog
04:02:41.134012
8.7966e-05
Selecting footprint 25
Writing footprinted catalog
04:02:41.395265
8.7966e-05
Selecting footprint 26
Writing footprinted catalog
04:02:41.684747
8.0455e-05
Selecting footprint 27
Writing footprinted catalog
04:02:41.961972
7.2027e-05
Selecting footprint 28
Writing footprinted catalog
04:02:42.253119
7.2027e-05
Selecting footprint 29
Writing footprinted catalog
04:02:42.551336
7.5397e-05
Selecting footprint 30
Writing footprinted catalog
04:02:42.871271
7.24e-05
Selecting footprint 31
Writing footprinted catalog
04:02:43.213290
7.24e-05
Selecting footprint 32
Writing footprinted catalog
04:02:43.564065
6.6024e-05
Selecting footprint 33
Writing footprinted catalog
04:02:43.936781
6.8661e-05
Selecting footprint 34
Writing footprinted catalog
04:02:44.402500
6.8661e-05
Selecting footprint 35
Writing footprinted catalog
04:02:44.802850
6.7186e-05
Selecting footprint 36
Writing footprinted catalog
04:02:

# make randoms for QSO

In [9]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
import glob
import os
from desimodel.io import fits

for j in range(1,2):
    for i in range(24,71):
 ###############################   
 ### change input files here ### 
        CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype2_{}.fits".format(i)
        d        = fits.open(CAT_file)
        length = d[1].header['NAXIS2']
    #    ra       = d[1].data['RA']
        ra = np.random.rand(length)*360
    #    dec      = d[1].data['DEC']
        dec = np.arccos(2*np.random.rand(length)-1)*90/np.pi*2-90
        zz       = d[1].data['Z']
        zz_rsd   = d[1].data['Z']+d[1].data['DZ']
        print(f"Selecting footprint {i} in set {j}")
        tiles = desimodel.io.load_tiles()
        point=foot.is_point_in_desi(tiles, ra, dec)
        index=[]
        index = np.where(point)
        ra_desi=ra[index]
        dec_desi=dec[index]
        zz_desi=zz[index]
        zz_rsd_desi=zz_rsd[index]
        print('Writing footprinted catalog')
    ############################################ 
    ### change the name of output files here ### 
        OUT_file ="/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_QSO_footprint_N_snapshot/random/lightcone_multibox_galtype2_{}_footprint_{}.ran".format(i,j)
        np.savetxt(OUT_file,np.c_[ra_desi,dec_desi,zz_desi,zz_rsd_desi])
        print(datetime.datetime.now().time())
        
# merge the output files above    
#######################################################    
### change the name to be merged to be consistent with the name above ### 
### change the name of the merged file ###        
    print(f"starting combine files {j}")
    for f in glob.glob("/global/homes/c/chuang/scratch_cori/UNITsims/lightcone_QSO_footprint_N_snapshot/random/lightcone_multibox_galtype2_*_footprint_{}.ran".format(j)):
             os.system("cat "+f+" >> /global/homes/c/chuang/scratch_cori/UNITsims/lightcone_QSO_footprint_N_snapshot/random/single_file/UNIT_lightcone_multibox_QSO_footprint_1xdata_{}.ran".format(j))
    print('finishing combine files')

Selecting footprint 24 in set 1
Writing footprinted catalog
15:35:27.252876
Selecting footprint 25 in set 1
Writing footprinted catalog
15:35:31.029031
Selecting footprint 26 in set 1
Writing footprinted catalog
15:35:34.520425
Selecting footprint 27 in set 1
Writing footprinted catalog
15:35:38.237612
Selecting footprint 28 in set 1
Writing footprinted catalog
15:35:42.470202
Selecting footprint 29 in set 1
Writing footprinted catalog
15:35:47.724852
Selecting footprint 30 in set 1
Writing footprinted catalog
15:35:51.366112
Selecting footprint 31 in set 1
Writing footprinted catalog
15:35:54.781954
Selecting footprint 32 in set 1
Writing footprinted catalog
15:35:59.711202
Selecting footprint 33 in set 1
Writing footprinted catalog
15:36:05.940110
Selecting footprint 34 in set 1
Writing footprinted catalog
15:36:10.307032
Selecting footprint 35 in set 1
Writing footprinted catalog
15:36:15.026773
Selecting footprint 36 in set 1
Writing footprinted catalog
15:36:20.189900
Selecting fo

# test

In [3]:
import desimodel.footprint as foot
import desimodel.io
import numpy as np
#import argparse                                                                                                                                                                  
import sys
import datetime
from desimodel.io import fits
import glob
import os
CAT_file="/global/homes/c/chuang/scratch_cori/UNITsims/desi_mock_multibox_v3/lightcone_multibox_galtype3_70.fits"
d        = fits.open(CAT_file)
d[0].header['NGALBOX']

3176887